In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from dotenv import find_dotenv, load_dotenv

load_dotenv(find_dotenv())

True

In [53]:
from farmwise_client import AgentClient

client = AgentClient(base_url="http://127.0.0.1:8082")

# await client.ainvoke("Do you have wifi?", user_id="+254748256530")
await client.ainvoke("Is it a boeing?", user_id="+254748256530")

ChatMessage(type='assistant', content="I'm unable to determine whether it's a Boeing plane. Let me transfer you to someone who can help with that.", tool_calls=[], tool_call_id=None, run_id=None, response_metadata={}, custom_data={})

In [55]:

from pydantic import BaseModel
import asyncio

from agents import Agent, Runner, function_tool

"""This demonstrates usage of the `previous_response_id` parameter to continue a conversation.
The second run passes the previous response ID to the model, which allows it to continue the
conversation without re-sending the previous messages.

Notes:
1. This only applies to the OpenAI Responses API. Other models will ignore this parameter.
2. Responses are only stored for 30 days as of this writing, so in production you should
store the response ID along with an expiration date; if the response is no longer valid,
you'll need to re-send the previous conversation history.
"""


class Weather(BaseModel):
    city: str
    temperature_range: str
    conditions: str


@function_tool
def get_weather(city: str) -> Weather:
    print("[debug] get_weather called")
    return Weather(city=city, temperature_range="14-20C", conditions="Sunny with wind.")


async def main():
    agent = Agent(
        name="Hello world",
        instructions="You are a helpful agent.",
        tools=[get_weather],
    )

    result = await Runner.run(agent, input="What's the weather in Tokyo?")
    print(result.final_output)
    # Brazil

    result = await Runner.run(
        agent,
        "What is the capital of that country?",
        previous_response_id=result.last_response_id,
    )
    print(result.final_output)
    # Brasilia


await main()



[debug] get_weather called
The weather in Tokyo is currently sunny with wind, and the temperature ranges from 14°C to 20°C.
The capital of Japan, where Tokyo is located, is Tokyo itself.
